# Dimensionnement d'une cuve

In [1]:
import numpy as np
import pandas as pd
import altair as alt
import tomli


## Objectif

On cherche ici à déterminer les dimensions d'une cuve de récupération d'eau de pluie telles que celle-ci puisse couvrir les besoins d'une habitation.
On dispose des données suivantes : 
- pluviométrie mensuelle moyenne ;
- consommation quotidienne moyenne.

## Méthode

On travail avec un pas de temps quotidien. 
Dans un premier temps on simule une pluviométrie quotidienne à partir des moyennes mensuelles.
Puis on considère une cuve de volume infini, qui chaque jour se remplit avec la pluie et se vide avec les consommations.
On en déduit enfin le volume minimal de la cuve pour absorber tous les besoins. 

On répète cette opération $n$ fois pour s'assurer que la cuve est suffisante en cas d'année sèche. 


## Modèle de pluviométrie

### Pluviométrie moyenne

Considérons la fiche climatique de Clermont-Ferrand. 

In [2]:
#Donnees
moyenne_clermont = pd.read_csv('../data/fiche_climat_clermont.csv')
#donnees = pd.read_csv('../data/donnees.csv', sep=',', index_col="name")
moyenne_clermont

,mois,hauteur,nb_jours,monthnum
0,janvier,26.6,6.4,1
1,fevrier,18.7,5.0,2
2,mars,26.1,6.5,3
3,avril,51.1,8.3,4
4,mai,66.5,9.4,5
5,juin,67.5,8.0,6
6,juillet,63.3,7.4,7
7,aout,62.0,7.5,8
8,septembre,57.5,6.7,9
9,octobre,48.8,7.8,10


Et visualisons-la :

In [3]:
alt.Chart(moyenne_clermont.reset_index()).mark_bar().encode(
    x = alt.X('monthnum:T', title='mois'),
    y = alt.Y('hauteur', title='Hauteur'),
    tooltip=['month:O', 'hauteur']
) | alt.Chart(moyenne_clermont.reset_index()).mark_bar(color='orange').encode(
    x = alt.X('month(month):N', title='Mois'), # , sort='mois')
    y = alt.Y('nb_jours', title='Nombre de jours de pluie'),
    tooltip=['mois', 'nb_jours']
)


alt.HConcatChart(...)

In [4]:
# Extraction des données
with open("../data/conf.toml", "rb") as f:
    toml_dict = tomli.load(f)
n_inhabitants = toml_dict['inhabitants']
surface_garden = toml_dict['surface_garden']
surface_roof = toml_dict['surface_roof']


A partir de ces données, on simule une année de pluviométrie.

### Pluviométrie simulée

On simule les pluies quotidiennes à partir des données mensuelles observées. 

Le nombre de jours de pluie dans le mois suit une loi normale centrée autour du nombre moyen observé.
A partir du nombre de jours de pluie simulé et la pluviométrie moyenne observée on calcule une pluviométrie moyenne pour chaque jour de pluie simulé. La pluviométrie simulée suit une loi normale centrée autour de cette valeur. 

In [5]:
from recuperation_eau_pluie import simulation


In [6]:
#initialisation
pluviometrie = pd.DataFrame({"precipitations" : np.zeros(365)}, index=pd.date_range("20230101", periods=365))

for month in range(len(moyenne_clermont['mois'])):
    #nombre de jours ou il pleut
    avg_days_obs = round(moyenne_clermont.loc[moyenne_clermont.index[month], 'nb_jours']) 
    avg_days = round(np.random.normal(avg_days_obs, avg_days_obs/4))
    
    # choix des jours
    start_date = pluviometrie.loc[pluviometrie.index.month == month+1].index.min()
    end_date =pluviometrie.loc[pluviometrie.index.month == month+1].index.max()
    rainy_days = simulation.generate_random_dates(start_date, end_date, avg_days)

    #hauteur tombee
    avg_rain = moyenne_clermont.loc[moyenne_clermont.index[month], 'hauteur']/avg_days
    obs_rain = simulation.generate_random_rain(avg_rain, avg_rain/4, avg_days)
    
    #remplacement des valeurs
    pluviometrie.loc[pluviometrie.index.isin(rainy_days), "precipitations"] = obs_rain

alt.Chart(pluviometrie.reset_index()).mark_bar().encode(
    x=alt.X('index:T', title="Jour de l'année"),
    y=alt.Y('precipitations:Q', title="precipitations"),
    tooltip=[alt.Tooltip("precipitations:Q")]
).properties(width=600).interactive()



alt.Chart(...)

In [7]:
##### Definition des fonctions

## Generer k dates aléatoires distinctes entre deux bornes
def generate_random_dates(start_date, end_date, k):
   date_range = (end_date - start_date).days + 1
   random_days = np.random.choice(date_range, size=k, replace=False)
   random_dates = pd.to_datetime(start_date) + pd.to_timedelta(random_days, unit='d')
   return random_dates

## Generer k hauteurs de pluie (loi normale autour d'une hauteur moyenne)
def generate_random_rain(mean_rain:float, sigma_rain:float, k:int):
   random_rain = np.random.normal(mean_rain, sigma_rain, size=k).round(1)
   return random_rain


In [8]:
#initialisation
pluviometrie = pd.DataFrame({"precipitations" : np.zeros(365)}, index=pd.date_range("20230101", periods=365))

for month in range(len(moyenne_clermont['mois'])):
    #nombre de jours ou il pleut
    avg_days_obs = round(moyenne_clermont.loc[moyenne_clermont.index[month], 'nb_jours']) 
    avg_days = round(np.random.normal(avg_days_obs, avg_days_obs/4))
    
    # choix des jours
    start_date = pluviometrie.loc[pluviometrie.index.month == month+1].index.min()
    end_date =pluviometrie.loc[pluviometrie.index.month == month+1].index.max()
    rainy_days = generate_random_dates(start_date, end_date, avg_days)

    #hauteur tombee
    avg_rain = moyenne_clermont.loc[moyenne_clermont.index[month], 'hauteur']/avg_days
    obs_rain = generate_random_rain(avg_rain, avg_rain/4, avg_days)
    
    #remplacement des valeurs
    pluviometrie.loc[pluviometrie.index.isin(rainy_days), "precipitations"] = obs_rain

alt.Chart(pluviometrie.reset_index()).mark_bar().encode(
    x=alt.X('index:T', title="Jour de l'année"),
    y=alt.Y('precipitations:Q', title="precipitations"),
    tooltip=[alt.Tooltip("precipitations:Q")]
).properties(width=600).interactive()



alt.Chart(...)

In [21]:
pluvio2 = simulation.generate_rainfall(moyenne_clermont["nb_jours"], moyenne_clermont["hauteur"])
alt.Chart(pluvio2.reset_index()).mark_bar().encode(
    x=alt.X('index:T', title="Jour de l'année"),
    y=alt.Y('rainfall:Q', title="precipitations"),
    tooltip=[alt.Tooltip("precipitations:Q")]
).properties(width=600).interactive()

alt.Chart(...)

## Consommation

### Typologie des consommations

On distingue ici trois sortes de besoins : 

* irrigation (potager)
* agricole (machines)
* domestiques

Dans un premier temps, nous ne considérerons que les besoins d'irrigation. 

### Besoins potager

Le potager à irriguer a une surface de $60 m^2$ . 

Pour simplifier les calculs, on ne prendra que trois valeurs de besoins en eau du potager : 

* D'octobre à avril le potager est au repos et n'a pas besoin d'eau.
* Entre juin et août : irrigation maximale.
* En mai et septembre, irrigation modérée (moitié) 

Pour simplifier, on considérera que l'irrigation est de $20L/m^2$ tous les trois jours *sans paillage* au plus fort de l'été. 
Le paillage divise ces besoins par trois. 



In [9]:
water_needs = pd.DataFrame({"potager" : np.zeros(365)}, index=pd.date_range("20230101", periods=365))
water_needs["20230501":"20230531"] = toml_dict['water_needs']/3
water_needs["20230901":"20230930"] = toml_dict['water_needs']/3
water_needs["20230601":"20230831"] = toml_dict['water_needs']

water_needs["potager_paillage"] = water_needs["potager"]/3

alt.Chart(water_needs.reset_index()).mark_line().encode(
    x=alt.X('index:T', title="Jour de l'année"),
    y=alt.Y('potager:Q', title="Consommation"),
    tooltip=[alt.Tooltip("conso_base:Q")]
).properties(width=600).interactive()



alt.Chart(...)

## Dimension de la cuve

Quels sont les besoins totaux ? 

In [10]:
print(np.sum(water_needs['potager']*100)) 


74926.33333333333


Le potager a donc besoin, en tout, de 82 $m^3$ au cours d'une année. 
Mais une partie de ces besoins est fournie par la pluie, lorsqu'elle tombe. 

In [11]:
df = pd.concat([pluviometrie, water_needs],axis=1)

df.loc[df['potager']>0, 'potager'] -=  df.loc[df['potager']>0, 'precipitations'] # quand il pleut les besoins sont reduits d'autant

# On cherche les endroits où il a plu davantage que les besoins.
negative_needs = df.loc[df['potager']<0, "potager"]

while(negative_needs.count()>0):
    # D'abord il n'y a plus de besoin pour le jour en question
    df.loc[df.index.isin(negative_needs.index), "potager"] = 0
    # l'excedent de pluie fournit les besoins du lendemain
    negative_needs.index = negative_needs.index + pd.Timedelta('1 day')
    df.loc[df.index.isin(negative_needs.index), "potager"] += negative_needs
    # On recalcule le nombre de jours où il a trop plu
    negative_needs = df.loc[df['potager']<0, "potager"]

alt.Chart(df.reset_index()).mark_line().encode(
    x = alt.X('index:T', title="Jour de l'année"),
    y = alt.Y('potager', title="Besoins en irrigation")
).properties(width=450).interactive()
# NB attention au potentiel dernier jour


alt.Chart(...)

In [12]:
# A partir des besoins et consommations par unité de surface,
# On obtient les besoins et consommations pour l'ensemble du potager. 
df['total_needs'] = df["potager"]*surface_garden
df['sum_apports'] = df['precipitations']*surface_roof

On accède donc, via la surface du potager et celle du toit, à l'ensemble des besoins et l'ensemble des apports quotidiens. 


### Dimension de la cuve

Simulons une cuve de volume infini, qui se remplit avec les précipitations et se vide via les utilisations, et suivons son volume. 

In [14]:
#initialisation 
cuve_infinie = pd.DataFrame({"volume" : np.zeros(365)}, index=pd.date_range("20230101", periods=365))
# remplissage au cours de l'annee
for i in range(len(cuve_infinie.index)-1):
    cuve_infinie["volume"].iloc[i+1] = cuve_infinie["volume"].iloc[i] + df['sum_apports'].iloc[i] - df['total_needs'].iloc[i]

alt.Chart(cuve_infinie.reset_index()).mark_line().encode(
    x=alt.X('index:T', title="Jour de l'année"), 
    y=alt.Y('volume', title="Volume de la cuve"), 
    tooltip=['volume', 'index']
).interactive()


alt.Chart(...)

La cuve, initialement vide, se remplit pendant les mois d'hiver. 
Elle se vide pendant les mois d'été, où les besoins sont plus importants.
Pour déterminer la taille minimale de la cuve, il faut que la quantité récoltée pendant l'hiver suffise exactement à combler le déficit de l'été. 
On fixe donc un volume maximal arbitraire, compris entre 0 et la pluviométrie cumulée pendant l'hiver. 
Puis la différence entre ce volume de remplissage et le volume minimal atteint pendant l'été donne la valeur minimale du volume de la cuve. 

In [17]:
#### obsolete

#Prenons d'abord une cuve vide
cuve = pd.DataFrame({"volume" : np.zeros(365)}, index=pd.date_range("20230101", periods=365))
#Prenons un volume limite pour la cuve, par exemple le remplissage à la fin de l'hiver.
vol_temoin=0
#vol_temoin = cuve_infinie.loc["20230501", "volume"]
# remplissage au cours de l'annee
for i in range(len(cuve.index)-1):
    cuve["volume"].iloc[i+1] = min(vol_temoin,cuve["volume"].iloc[i] + df['sum_apports'].iloc[i] - df['total_needs'].iloc[i])
#Le minimum atteint par la cuve donne le volume nécessaire
#volume minimal
volume_minimal = cuve["20230501":"20231031"]["volume"].min()
vol_necessaire = vol_temoin - volume_minimal
print('Volume minimal de la cuve : ',vol_necessaire)
print(cuve["20230101":"20231231"]["volume"].min())
alt.Chart(cuve.reset_index()).mark_line().encode(
    x=alt.X('index:T', title="Jour de l'année"), 
    y=alt.Y('volume', title="Volume de la cuve"), 
    tooltip=['volume', 'index']
).interactive()

Volume minimal de la cuve :  3601.7999999999993
-3601.7999999999993


alt.Chart(...)

In [18]:
var = simulation.min_reservoir_volume(df['sum_apports'], df['total_needs'])
var

3601.7999999999993

Vérification avec le nouveau volume de cuve : 

In [ ]:
#Nouvelle itération 
for i in range(len(cuve.index)-1):
    cuve["volume"].iloc[i+1] = min(vol_necessaire,cuve["volume"].iloc[i] + df['sum_apports'].iloc[i] - df['total_needs'].iloc[i])

#Illustration
alt.Chart(cuve.reset_index()).mark_line().encode(
    x=alt.X('index:T', title="Jour de l'année"), 
    y=alt.Y('volume', title="Volume de la cuve"), 
    tooltip=['volume', 'index']
).interactive()


alt.Chart(...)

Vérifions le comportement d'une telle cuve. 

In [ ]:
print(vol_necessaire)

4251.5999999999985


Let's run it a thousand times ! 

In [22]:
volumes = pd.DataFrame({"volume" : np.zeros(100)})
for index in range(len(volumes)):
    pluviosim = simulation.generate_rainfall(moyenne_clermont["nb_jours"], moyenne_clermont["hauteur"])
    volmin=simulation.min_reservoir_volume(surface_roof*pluviosim, df['total_needs'])
    volumes.loc[index]=volmin

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

## PLUS ULTRA

Les projets pour la suite 

### tentative d'appel de fonction dans un dossier parent

La première étape est de mettre les fonctions dans la librairie pour y faire appel. 

Il faut aussi faire en priorité la version "paillage", qui est la plus réaliste. 

Puis stocker les résultats de n simulations.

Enfin évaluer les hypothèses de génération des pluies, pour faire apparaître par exemple des sécheresses. 

In [ ]:
# Il faut que tu importes depuis un endroit connu
# Ton projet définit la _library_ `recuperation_eau_pluie`,
# et l'installe automatiquement de façon éditable
# (on en discute à l'occasion si ça t'intéresse)
from recuperation_eau_pluie import simulation


In [ ]:
simulation.test_function()

Hello World ! 4


In [ ]:
import tomli

with open("../data/conf.toml", "rb") as f:
    toml_dict = tomli.load(f)
n_inhabitants = toml_dict['inhabitants']
surface_garden = toml_dict['surface_garden']
surface_roof = toml_dict['surface_roof']


In [ ]:
toml_dict

{'surface_garden': 60,
 'surface_roof': 200,
 'mean_human_daily_consumption': 0.15,
 'inhabitants': 6}

6